In [4]:
link = 'https://learn.dict.naver.com/wordbook/enkodict/#/my/cards?wbId=1b871a04bbe2435c97b4623010c42aac&qt=0&st=0&name=%ED%95%B4%EC%BB%A4%EC%8A%A4%EC%8B%A4%EB%AA%A8%20test1%20voca&tab=list'
wordbook_name = 'test1 voca'

In [2]:
!pip install selenium

#크롬드라이버 자동다운로드
!pip install webdriver_manager
from selenium import webdriver
from selenium.webdriver.common.by import By

#크롬드라이버 자동다운로드 사용시
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time

#크롬드라이버 자동다운로드
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.get(link)

# 여기서 수동로그인 하기

In [5]:
total_word_data=[]
while 1:
    time.sleep(1)
    word_card_list = driver.find_elements(By.CLASS_NAME, 'card_word')
    for word_card in word_card_list: # 각각의 단어카드에 대해...

        ##단어와 발음 추출
        word = (word_card.find_element(By.CLASS_NAME, 'title').text).replace('·', '')
        try: pronunciation = word_card.find_element(By.CLASS_NAME, 'pronounce').text
        except: pronunciation = ''

        ##의미 추출
        meaning_list = []
        meaning_blocks = word_card.find_elements(By.CLASS_NAME, 'item_mean')
        for meaning_block in meaning_blocks:
            meaning = meaning_block.find_element(By.CLASS_NAME, 'cont').text
            i = meaning.find(' ')
            clas = meaning[:i]
            mean = meaning[i+1:]
            meaning_list.append(clas + '| ' +  mean)

        total_word_data.append([word, pronunciation, meaning_list])
    
    print(f"crawled this page. len = {len(total_word_data)}")
    
    #다음 페이지 가능하면 넘어가고, 아니면 break
    next_button = driver.find_element(By.CLASS_NAME, 'btn_next')
    class_list = next_button.get_attribute('class').split(' ')
    if 'disabled' in class_list: break
    next_button.click()
        
print('done')

!pip install python-docx



import random
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml
from docx.shared import Inches
from docx.shared import Cm


# 문서 생성
doc = Document()

# 문서의 섹션 가져오기
section = doc.sections[0]

# 여백 조정 (인치 단위)
section.top_margin = Inches(0.5)
section.bottom_margin = Inches(0.5)
section.left_margin = Inches(0.5)
section.right_margin = Inches(0.5)

# 표 생성
table = doc.add_table(rows=len(total_word_data), cols=2)



# 폰트 설정
style = doc.styles['Normal']
font = style.font
font.name = 'Georgia'

# 표 스타일 설정
table.style = 'Table Grid'

# 각 행의 데이터 입력
for i in range(len(total_word_data)):
    datum = total_word_data[i]
    ith_row = table.rows[i].cells
    
    # 1열
    cell = ith_row[0]
    cell.text = datum[0]
    try:
        # 첫 번째 줄의 폰트 크기를 14포인트로 설정
        cell.paragraphs[0].runs[0].font.size = Pt(14)
        # 둘째 줄의 폰트 크기를 8포인트로 설정하기 위해 두 번째 Run 객체 생성
        second_run = cell.paragraphs[0].add_run('\n' + datum[1])
        second_run.font.size = Pt(8)
    except:
        pass
    # 우측 정렬 설정
    cell.paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT
    
    
    # 2열
    cell = ith_row[1]
    cell.text = '\n'.join(datum[2])

# 각 행과 행 사이의 구분선 설정 (실선)
for row in table.rows:
    tr = row._tr
    trPr = tr.get_or_add_trPr()
    tblBorders = parse_xml(r'<w:tblBorders xmlns:w="http://schemas.openxmlformats.org/wordprocessingml/2006/main">'
                            '<w:top w:val="single" w:sz="4" w:space="0" w:color="auto"/>'
                            '<w:bottom w:val="single" w:sz="4" w:space="0" w:color="auto"/>'
                            '<w:left w:val="single" w:sz="4" w:space="0" w:color="auto"/>'
                            '<w:right w:val="single" w:sz="4" w:space="0" w:color="auto"/>'
                            '</w:tblBorders>')
    trPr.append(tblBorders)

# 표의 너비 설정
table.columns[0].width = Cm(6.5)  # 1열의 너비를 5cm로 설정
table.columns[1].width = Cm(12.5)  # 2열의 너비를 4cm로 설정 (예시)     
# 문서 저장
doc.save(wordbook_name + ' 암기용.docx')
print('암기용 생성완료')

##################################################################
random.shuffle(total_word_data)



# 문서 생성
doc = Document()

# 문서의 섹션 가져오기
section = doc.sections[0]

# 여백 조정 (인치 단위)
section.top_margin = Inches(0.5)
section.bottom_margin = Inches(0.5)
section.left_margin = Inches(0.5)
section.right_margin = Inches(0.5)

# 표 생성
table = doc.add_table(rows=len(total_word_data), cols=3)



# 폰트 설정
style = doc.styles['Normal']
font = style.font
font.name = 'Georgia'

# 표 스타일 설정
table.style = 'Table Grid'

# 각 행의 데이터 입력
for i in range(len(total_word_data)):
    datum = total_word_data[i]
    ith_row = table.rows[i].cells
    
    # 1열 (우측 정렬)
    cell = ith_row[0]
    cell.text = datum[0]
    cell.paragraphs[0].alignment = WD_PARAGRAPH_ALIGNMENT.RIGHT
    
    # 3열
    cell = ith_row[2]
    cell.text = '\n'.join(datum[2])

# 각 행과 행 사이의 구분선 설정 (실선)
for row in table.rows:
    tr = row._tr
    trPr = tr.get_or_add_trPr()
    tblBorders = parse_xml(r'<w:tblBorders xmlns:w="http://schemas.openxmlformats.org/wordprocessingml/2006/main">'
                            '<w:top w:val="single" w:sz="4" w:space="0" w:color="auto"/>'
                            '<w:bottom w:val="single" w:sz="4" w:space="0" w:color="auto"/>'
                            '<w:left w:val="single" w:sz="4" w:space="0" w:color="auto"/>'
                            '<w:right w:val="single" w:sz="4" w:space="0" w:color="auto"/>'
                            '</w:tblBorders>')
    trPr.append(tblBorders)

# 표의 너비 설정
table.columns[0].width = Cm(6.5)  # 1열의 너비를 5cm로 설정
table.columns[1].width = Cm(12.5)  # 2열의 너비를 4cm로 설정 (예시)    

for row in table.rows:
    cell = row.cells[2]  # 3열의 셀 선택
    cell.paragraphs[0].runs[0].font.size = Pt(8)  # 폰트 크기를 8pt로 설정 (예시)

# 문서 저장
doc.save(wordbook_name + ' 시험용.docx')
print('시험용 생성완료')

crawled this page. len = 20
crawled this page. len = 40
crawled this page. len = 60
crawled this page. len = 79
done


암기용 생성완료
시험용 생성완료
